## Практика 12


### Задание 1. Выгрузка данных и их предобработка

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
import time
import catboost as cb

In [120]:
data = pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [122]:
data.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [123]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [124]:
data.drop_duplicates(inplace=True)
data = data.reset_index(drop=True)

In [125]:
target = data['Outcome']
predictors = data.drop('Outcome', axis=1)

In [126]:
x_train, x_test, y_train, y_test = train_test_split(predictors, target, train_size=0.8, shuffle=True)

### Задание 2. Реализация баггинга

In [127]:
random_forest = RandomForestRegressor()
params_grid = {
    "max_depth": np.arange(4, 32, 4),
    "min_samples_leaf": np.arange(3, 15, 3),
    "min_samples_split": np.arange(3, 15, 3)
}
grid_search_random_forest = GridSearchCV(estimator=random_forest,
                                         param_grid=params_grid,
                                         scoring='r2',
                                         cv=4)

start_time = time.time()
grid_search_random_forest.fit(x_train, y_train)
nice_tree_time = time.time() - start_time

In [129]:
best_model = grid_search_random_forest.best_estimator_

In [130]:
y_predict_nice_forest = best_model.predict(x_test)

In [131]:
sk_r_f_s = r2_score(y_test, y_predict_nice_forest)
sk_r_f_s

0.34656576110422266

### Задание 3. Реализация бустинга

In [137]:
model_catboost_clf = cb.CatBoostRegressor(iterations=40000, task_type='CPU')

In [138]:
start_time = time.time()
model_catboost_clf.fit(x_train, y_train)
catboost_time = time.time() - start_time

Learning rate set to 0.001889
0:	learn: 0.4735998	total: 2.34ms	remaining: 1m 33s
1:	learn: 0.4733370	total: 4.29ms	remaining: 1m 25s
2:	learn: 0.4731278	total: 6.52ms	remaining: 1m 26s
3:	learn: 0.4729210	total: 9.22ms	remaining: 1m 32s
4:	learn: 0.4727087	total: 11.1ms	remaining: 1m 28s
5:	learn: 0.4724661	total: 13.2ms	remaining: 1m 27s
6:	learn: 0.4722787	total: 15.3ms	remaining: 1m 27s
7:	learn: 0.4720397	total: 17.5ms	remaining: 1m 27s
8:	learn: 0.4718154	total: 20.8ms	remaining: 1m 32s
9:	learn: 0.4715938	total: 25.9ms	remaining: 1m 43s
10:	learn: 0.4713666	total: 29.2ms	remaining: 1m 46s
11:	learn: 0.4711318	total: 31.2ms	remaining: 1m 43s
12:	learn: 0.4708798	total: 33.2ms	remaining: 1m 41s
13:	learn: 0.4706936	total: 35.7ms	remaining: 1m 42s
14:	learn: 0.4704794	total: 38.1ms	remaining: 1m 41s
15:	learn: 0.4702299	total: 40.6ms	remaining: 1m 41s
16:	learn: 0.4700366	total: 42.4ms	remaining: 1m 39s
17:	learn: 0.4698084	total: 44.4ms	remaining: 1m 38s
18:	learn: 0.4695792	total

In [134]:
y_predict_catboost = model_catboost_clf.predict(x_test)
cb_s = r2_score(y_test, y_predict_catboost)
cb_s

0.2718003881259624

### Задание 4. Сравнение алгоритмов

In [139]:
time_data = pd.DataFrame({'Random Forest Time': [nice_tree_time],
                     'Catboost Time': [catboost_time]})
time_data

,Random Forest Time,Catboost Time
0,159.703276,93.208385


In [140]:
score_data = t_df = pd.DataFrame({'Random Forest Score': [sk_r_f_s],
                            'Catboost Score': [cb_s]})
score_data

,Random Forest Score,Catboost Score
0,0.346566,0.2718
